In [ ]:
## Workflow - Documentation for Application

#### Registrera ny medlem

INSERT INTO Members (Username, Email, PasswordHash, Gender, Birthdate, RelationshipStatus, SexualOrientation, LocationRegion, LocationCity)
VALUES ('SkåneRomeo', 'romeo@example.com', 'hashed_pw_1', 'Male', '1990-03-22', 'Single', 'Straight', 'Skåne', 'Lund');

#### Skapa profil

INSERT INTO Profiles (MemberID, DisplayName, Bio, Interests, ProfilePictureURL)
VALUES (LAST_INSERT_ID(), 'SkåneRomeo', 'Hej! Jag är en glad kille från Skåne som älskar att resa och träffa nya människor.', 'Resor, Musik, Sport', 'http://example.com/profiles/skaneromeo.jpg');

#### Lägg till intressen

INSERT INTO Interests (InterestName)
VALUES ('Hiking'), ('Cooking'), ('Photography'), ('Gaming');

#### Koppla intressen till medlemmen

INSERT INTO MemberInterests (MemberID, InterestID)
VALUES (1, 1), (1, 2); -- SkåneRomeo gillar Hiking och Cooking

#### Matcha med annan medlem

INSERT INTO Matches (MemberA, MemberB, MatchDate)
VALUES (1, 2, NOW()); -- SkåneRomeo matchar med medlem med ID 2

#### Skicka ett meddelande

INSERT INTO Messages (SenderID, ReceiverID, MessageText, SentAt)
VALUES (1, 2, 'Hej! Kul att matcha med dig. Vill du ta en fika någon dag?', NOW());

#### Uppdatera profil

UPDATE Profiles
SET Bio = 'Hej! Jag är en glad kille från Skåne som älskar
    att resa, träffa nya människor och fotografera.'
    WHERE MemberID = 1;
-- SkåneRomeo uppdaterar sin bio

#### Visa meddelanden mellan två medlemmar (ADMIN)

SELECT Sender.Username AS FromUser, Receiver.Username AS ToUser, M.MessageText, M.SentAt
FROM Messages
JOIN Members AS Sender ON M.SenderID = Sender.MemberID
JOIN Members AS Receiver ON M.ReceiverID = Receiver.MemberID
WHERE (M.SenderID = 1 AND M.ReceiverID = 2) OR (M.SenderID = 2 AND M.ReceiverID = 1)
ORDER BY M.SentAt DESC;
-- Visa alla meddelanden mellan SkåneRomeo och medlem med ID 2

#### Radera medlem (ADMIN)

DELETE FROM Members
WHERE MemberID = 1; 
-- Radera SkåneRomeo och alla relaterade data (cascading delete)


#### Medlemmar
INSERT INTO Members (Username, Email, PasswordHash, Gender, Birthdate, RelationshipStatus, SexualOrientation, LocationRegion, LocationCity)
VALUES 
('SkåneRomeo', 'romeo@example.com', 'hashedpw', 'Female', '1990-03-22', 'Single', 'Straight', 'Skåne', 'Lund'),
('GothenburgGina', 'gina@example.com', 'hashedpw', 'Female', '1988-07-15', 'Single', 'Bisexual', 'Västra Götaland', 'Gothenburg'),
('StockholmSam', 'sam@example.com', 'hashedpw', 'Female', '1992-11-30', 'In a relationship', 'Not specified', 'Stockholm', 'Stockholm');

#### Intressen
INSERT INTO Interests (InterestName)
VALUES ('Hiking'), ('Cooking'), ('Photography'), ('Gaming');

#### Kopplingar
INSERT INTO MemberInterests (MemberID, InterestID)
VALUES 
(1, 1), (1, 2), -- Romeo gillar Hiking och Cooking
(2, 2), (2, 3), -- LoveBird gillar Cooking och Photography
(3, 3), (3, 4); -- FotoFan gillar Photography och Gaming

#### Profiler
INSERT INTO Profiles (MemberID, Bio, ProfilePictureURL, LookingFor)
VALUES 
(1, 'Älskar natur och pasta.', 'https://example.com/romeo.jpg', 'Seriöst förhållande'),
(2, 'Fotointresserad och matälskare.', 'https://example.com/lovebird.jpg', 'Vänskap'),
(3, 'Letar efter någon att spela med.', 'https://example.com/fotofan.jpg', 'Avslappnat');


#### Matchningslogik (baserat på intressen och plats)
SELECT DISTINCT m1.MemberID AS UserID, m2.MemberID AS PotentialMatchID, m2.Username
FROM MemberInterests mi1
JOIN MemberInterests mi2 ON mi1.InterestID = mi2.InterestID AND mi1.MemberID != mi2.MemberID
JOIN Members m1 ON mi1.MemberID = m1.MemberID
JOIN Members m2 ON mi2.MemberID = m2.MemberID
WHERE m1.LocationRegion = m2.LocationCity;

#### Rekommenderade profiler (VIEW)
CREATE VIEW RecommendedProfiles AS
SELECT m1.MemberID AS ViewerID, m2.MemberID AS SuggestedID, m2.Username, i.InterestName
FROM MemberInterests mi1
JOIN MemberInterests mi2 ON mi1.InterestID = mi2.InterestID AND mi1.MemberID != mi2.MemberID
JOIN Members m1 ON mi1.MemberID = m1.MemberID
JOIN Members m2 ON mi2.MemberID = m2.MemberID
JOIN Interests i ON mi2.InterestID = i.InterestID
WHERE m1.LocationRegion = m2.LocationRegion;

#### Meddelandehantering

INSERT INTO Messages (SenderID, ReceiverID, MessageText, SentAt)
VALUES (1, 2, 'Hej! Din profil fångade verkligen mitt intresse 😊');

SELECT Sender.Username AS FromUser, Receiver.Username AS ToUser, MessageText, SentAt
FROM Messages
JOIN Members Sender ON Messages.SenderID = Sender.MemberID
JOIN Members Receiver ON Messages.ReceiverID = Receiver.MemberID
ORDER BY SentAt;

#### Blockeringar

INSERT INTO Blocks (BlockerID, BlockedID)
VALUES (2, 1); -- LoveBird blockerar Romeo

-- Visa blockerade användare för LoveBird
SELECT b.BlockedID, m.Username
FROM Blocks b
JOIN Members m ON b.BlockedID = m.MemberID
WHERE b.BlockerID = 2;

#### Likes och matchningar

INSERT INTO Likes (LikerID, LikedID)
VALUES (1, 2), (2, 1); -- Romeo och LoveBird gillar varandra

-- Skapa match om båda har gillat varandra
INSERT INTO Matches (MemberA, MemberB)
SELECT l1.LikerID, l1.LikedID
FROM Likes l1
JOIN Likes l2 ON l1.LikerID = l2.LikedID AND l1.LikedID = l2.LikerID
WHERE NOT EXISTS (
  SELECT 1 FROM Matches
  WHERE (MemberA = l1.LikerID AND MemberB = l1.LikedID)
     OR (MemberA = l1.LikedID AND MemberB = l1.LikerID)
);

### Statistik
#### Antal medlemmar per stad

SELECT LocationCity, COUNT(*) AS MemberCount
FROM Members
GROUP BY LocationCity;

#### Populära intressen
SELECT i.InterestName, COUNT(mi.MemberID) AS Popularity
FROM MemberInterests mi
JOIN Interests i ON mi.InterestID = i.InterestID
GROUP BY i.InterestName
ORDER BY Popularity DESC;


